In [5]:
import tmm
import numpy as np
import matplotlib.pyplot as plt

In [6]:
# tests

def coh_tmm_test() -> bool:
    pass

In [1]:
from numpy import NaN, zeros, exp, inf, cos, sin, conj
import scipy as sp

def make_2x2_array(a, b, c, d, dtype=float):
    """
    Makes a 2x2 numpy array of [[a,b],[c,d]]

    Same as "numpy.array([[a,b],[c,d]], dtype=float)", but ten times faster
    """
    my_array = np.empty((2,2), dtype=dtype)
    my_array[0,0] = a
    my_array[0,1] = b
    my_array[1,0] = c
    my_array[1,1] = d
    return my_array

def inverse_coh_tmm(n_list, d_list, lam_vac, reflected_power, transmitted_power):
    # convert lists to numpy array if they are not already
    n_list = np.array(n_list)
    d_list = np.array(d_list, dtype=float)
    if np.isnan(n_list).any():
        print(":3")
        return _inverse_coh_tmm_index(n_list, d_list, 0, lam_vac, reflected_power, transmitted_power)
    else:
        return _inverse_coh_tmm_thickness(n_list, d_list, 0, lam_vac, reflected_power, transmitted_power)
    pass

def left_list_snell(n_list, th_0):
    angles = sp.arcsin(n_list[-1]*np.sin(th_0) / n_list)

# to compute the left and right substacks, where the layer in between the two substacks is our desired layer. 
def _coh_tmm_substack(n_list, d_list, lam_vac, th_list, pol, is_left):
    
    num_layers = len(n_list)

    if num_layers == 1:
        return make_2x2_array(1,0,0,1)

    kz_list = 2*np.pi*n_list*np.cos(th_list)/lam_vac
    olderr = np.seterr(invalid='ignore')
    delta = kz_list*d_list
    np.seterr(**olderr)
    
    t_list = zeros((num_layers, num_layers), dtype=complex)
    r_list = zeros((num_layers, num_layers), dtype=complex)
    for i in range(num_layers-1):
        t_list[i,i+1] = tmm.interface_t(pol, n_list[i], n_list[i+1],
                                    th_list[i], th_list[i+1])
        r_list[i,i+1] = tmm.interface_r(pol, n_list[i], n_list[i+1],
                                    th_list[i], th_list[i+1])
        
    M_list = zeros((num_layers, 2, 2), dtype=complex)

    """
    Quick explanation, since this is the only part that differs from tmm

    In coh_tmm, they exclude both the first and last layer (since those are the medium). 
    Here, the first and last layers are not necessarily te medium. Thus, I have to exclude the 
    first layer when computing the left-hand-side transfer matrix and exclude the last layer when 
    computing the right-hand-side transfer matrix. 
    """

    if is_left:
        for i in range(1, num_layers):
            M_list[i] = (1/t_list[i,i+1]) * np.dot(
                make_2x2_array(exp(-1j*delta[i]), 0, 0, exp(1j*delta[i]),
                            dtype=complex),
                make_2x2_array(1, r_list[i,i+1], r_list[i,i+1], 1, dtype=complex))
        Mtilde = make_2x2_array(1, 0, 0, 1, dtype=complex)
        for i in range(1, num_layers):
            Mtilde = np.dot(Mtilde, M_list[i])
        Mtilde = np.dot(make_2x2_array(1, r_list[0,1], r_list[0,1], 1,
                                   dtype=complex)/t_list[0,1], Mtilde) # only need on LHS
    else: 
        for i in range(0, num_layers - 1):
            M_list[i] = (1/t_list[i,i+1]) * np.dot(
                make_2x2_array(exp(-1j*delta[i]), 0, 0, exp(1j*delta[i]),
                            dtype=complex),
                make_2x2_array(1, r_list[i,i+1], r_list[i,i+1], 1, dtype=complex))
        Mtilde = make_2x2_array(1, 0, 0, 1, dtype=complex)
        for i in range(0, num_layers - 1):
            Mtilde = np.dot(Mtilde, M_list[i])

    return Mtilde

        
    

def _inverse_coh_tmm_index(n_list, d_list, th_0, lam_vac, reflected_power, transmitted_power):
    nan_index = np.where(np.isnan(n_list))[0][0]
    th_list = tmm.list_snell(n_list, th_0)
    print(nan_index)
    left_side_n = n_list[:nan_index]
    right_side_n = n_list[nan_index+1:]
    left_side_d = d_list[:nan_index]
    right_side_d = d_list[nan_index+1:]
    left_side_th = th_list[:nan_index]
    right_side_th = th_list[nan_index+1:]

    left_side_M = _coh_tmm_substack(left_side_n, left_side_d, lam_vac, left_side_th, 's', True)
    right_side_M = _coh_tmm_substack(right_side_n, right_side_d, lam_vac, right_side_th, 's', False)

    left_side_M_abs = np.abs(left_side_M)
    right_side_M_abs = np.abs(right_side_M)

    """
    LM \dot M \dot RM = Mtilde
    """

    # solve for parameters of Mtilde and solve for the two interface matrices
    # R = abs(r)**2 <=> we're given R, and we have to solve for r
    # T = abs(t**2) * C
    # instead, we first consider only the amplitude of R and T

    n_f = n_list[-1]
    n_i = n_list[0]
    th_f = th_list[-1]
    th_i = th_list[0]
    # if pol = 's':
    #     c = (((n_f*cos(th_f)).real) / (n_i*cos(th_i)).real)
    # elif pol = 'p':
    #     c = (((n_f*conj(cos(th_f))).real) /
    #                             (n_i*conj(cos(th_i))).real)
    c = (n_f*cos(th_f)).real / (n_i*cos(th_i)).real 
    mag_R = np.sqrt(reflected_power)
    mag_T = np.sqrt(np.abs(transmitted_power/c))
    temp = mag_R/mag_T
    Mtilde = make_2x2_array((1/mag_T), (temp), (temp), (1/mag_T))

    Mtilde = np.dot(Mtilde, np.linalg.inv(right_side_M_abs))
    Mtilde = np.dot(np.linalg.inv(left_side_M_abs), Mtilde) # this is the identity matrix if it is the first layer

    refractive_index_quotient =  Mtilde[0,0]/Mtilde[0,1]
    pass

def _inverse_coh_tmm_thickness(n_list, d_list, lam_vac, reflected_power, transmitted_power):
    pass

In [20]:
inverse_coh_tmm([1, NaN, 3.5, 1], [inf, 20, 100, inf], 1000, 0.4, 0.3)

:3
1
[[1. 0.]
 [0. 1.]] [[-0.37786195-0.52008235j -0.2099233 -0.28893464j]
 [-0.2099233 +0.28893464j -0.37786195+0.52008235j]]
